<a href="https://colab.research.google.com/github/iamdsc/humor_detection/blob/master/humor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Humor Detection in One-liners

In [29]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 7091-600A

 Directory of C:\Users\steen\OneDrive\Documents\Github\TherapyNow\research\datasets\suicidal-ideation-datasets

04/26/2020  03:42 AM    <DIR>          .
04/26/2020  03:42 AM    <DIR>          ..
04/25/2020  09:32 PM    <DIR>          .ipynb_checkpoints
04/25/2020  07:54 PM            17,396 annotated_data0.csv
04/25/2020  07:58 PM            18,930 annotated_data1.csv
04/25/2020  07:54 PM            18,587 annotated_data2.csv
04/25/2020  07:54 PM            18,295 annotated_data3.csv
04/25/2020  07:54 PM            18,291 annotated_data4.csv
04/25/2020  07:54 PM            17,706 annotated_data5.csv
04/25/2020  07:54 PM            18,070 annotated_data6.csv
04/25/2020  07:54 PM            17,609 annotated_data7.csv
04/25/2020  07:54 PM            18,247 annotated_data8.csv
04/25/2020  07:54 PM            19,789 annotated_data9.csv
04/25/2020  09:30 PM    <DIR>          datasets
04/25/2020  09:31 PM    <DIR>          h

In [30]:
import pandas as pd


humour = pd.read_pickle('datasets/humorous_oneliners.pickle')
proverb = pd.read_pickle('datasets/proverbs.pickle')
wiki = pd.read_pickle('datasets/wiki_sentences.pickle')
long_humour = pd.read_pickle('datasets/oneliners_incl_doubles.pickle')
reuters = pd.read_pickle('datasets/reuters_headlines.pickle')

In [31]:
print(len(humour), len(proverb), len(wiki), len(reuters))

5251 1019 5251 5243


In [32]:
# building dataframe from the data
# giving humourous sentence positive class and rest negative class
humour_record = [(sent, 1) for sent in humour]
proverb_record = [(sent, 0) for sent in proverb]
wiki_record = [(sent, 0) for sent in wiki]
reuter_record = [(sent, 0) for sent in reuters]

In [33]:
import random


pos_record = humour_record
neg_record = proverb_record + wiki_record + reuter_record
columns = ['sentence', 'class']

# Randomly picking len(pos_records) records from negative class
random.shuffle(neg_record)
neg_record = neg_record[:len(pos_record)]

# Build the dataframe
df_record = pos_record + neg_record 
df = pd.DataFrame(df_record, columns=columns)

# shuffle the dataframe and reset the index
df = df.sample(frac=1).reset_index(drop=True)
print(df.head())

                                            sentence  class
0  Men look at a woman's behind and go "Oh! What ...      1
1  I used to be a wrestler, I would get a guy in ...      1
2            Do you need a silencer to shoot a mime?      1
3  I used to play the triangle in a reggae band b...      1
4  What happened to the fly that sat on the toile...      1


In [34]:
df['class'].value_counts()

1    5251
0    5251
Name: class, dtype: int64

In [36]:
# Creating word vectors
from gensim.models import FastText

text=[]
for sentence,_ in df_record:
  sent_word_list = [word for word in sentence.lower().split()]
  text.append(sent_word_list)

w2v = Word2Vec(text, min_count=1)

In [38]:
# building list of sentence vectors
vect_record=[]
for i in range(len(df['sentence'])):
  sent = df['sentence'][i]
  if len(sent)!=0:
    sent_vect = sum([w2v[w] for w in sent.lower().split()])/(len(sent.split())+0.001)
  else:  
    sent_vect = np.zeros((100,))
  vect_record.append(sent_vect) 

C:\Users\steen\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [39]:
X = pd.DataFrame(vect_record, columns=range(100))
print(X.head())
print(X.shape)

         0         1         2         3         4         5         6   \
0 -0.658298  0.949079 -0.041581  0.371437 -0.478547  0.044556  0.979343   
1 -0.827007  1.162999 -0.092464  0.397053 -0.714838 -0.070122  1.202632   
2 -0.780032  1.113149 -0.001697  0.495354 -0.429449  0.204053  1.150570   
3 -0.787824  1.111206 -0.093086  0.376728 -0.688702 -0.070569  1.146196   
4 -0.656605  0.943812 -0.057467  0.358178 -0.507506  0.021298  0.970246   

         7         8         9   ...        90        91        92        93  \
0 -0.236939 -0.800766 -0.347240  ... -0.132711 -1.045771 -0.065090 -0.735672   
1 -0.249733 -0.940331 -0.466606  ... -0.074358 -1.245247 -0.118414 -0.956603   
2 -0.311879 -0.981661 -0.385503  ... -0.277987 -1.294405 -0.046621 -0.796837   
3 -0.239507 -0.892008 -0.439740  ... -0.066334 -1.181674 -0.112064 -0.915533   
4 -0.227570 -0.783501 -0.344436  ... -0.113678 -1.030849 -0.071326 -0.745172   

         94        95        96        97        98        99  
0 -0

In [40]:
y = df['class']
print(type(y))

<class 'pandas.core.series.Series'>


In [41]:
# Performaing train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [42]:
print('Training Data: ', X_train.shape, y_train.shape)
print('Test Data: ', X_test.shape, y_test.shape)

Training Data:  (8401, 100) (8401,)
Test Data:  (2101, 100) (2101,)


In [43]:
from sklearn.svm import SVC

# fit the SVM model with linear Kernel
svm = SVC(kernel='linear', random_state=1, C=1.0)
svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=1,
    shrinking=True, tol=0.001, verbose=False)

In [44]:
# performing prediction in test samples
y_pred = svm.predict(X_test)
print(y_pred[:5])

[0 1 0 0 0]


In [45]:
# Calculating Classification Accuracy
from sklearn.metrics import accuracy_score

print('Test Accuracy Score: ',accuracy_score(y_pred, y_test))

Test Accuracy Score:  0.829128986197049


In [46]:
# Calculating Cross Validation Accuracy
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(estimator=svm, X=X_train, y=y_train, cv=10, n_jobs=-1)
print('CV accuracy: %.3f +/- %.3f'%(np.mean(scores),np.std(scores)))

CV accuracy: 0.814 +/- 0.019


In [47]:
# Using Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train, y_train)

# performing prediction on test samples
y_pred = lr.predict(X_test)

print('Test Accuracy score: ', accuracy_score(y_pred, y_test))

scores = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=10, n_jobs=-1)
print('CV accuracy: %.3f +/- %.3f'%(np.mean(scores),np.std(scores)))

C:\Users\steen\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test Accuracy score:  0.8810090433127082
CV accuracy: 0.868 +/- 0.014


In [48]:
from sklearn.externals import joblib

joblib.dump(lr, 'humor_detection_linreg.pkl') 

['humor_detection_linreg.pkl']